[Reference](https://medium.com/@oleg.kazanskyi/trading-analytics-with-financial-kpis-33df3b1cff24)

In [1]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 985 kB 5.0 MB/s 
     |████████████████████████████████| 358 kB 61.2 MB/s 
     |████████████████████████████████| 139 kB 44.9 MB/s 
     |████████████████████████████████| 4.1 MB 53.5 MB/s 
     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 58 kB 5.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.11 which is incompatible.


In [2]:
import pandas as pd
import datetime
from datetime import date

def get_vix(last_date = date.today(), historical_days = 1450):
    historical_date = last_date-datetime.timedelta(days=historical_days)
    
    url="https://cdn.cboe.com/api/global/us_indices/daily_prices/VIX_History.csv"
    response=pd.read_csv(url)
    
    response["DATE"]=pd.to_datetime(response["DATE"])
    response.rename(columns = {"DATE":"date"}, inplace = True)
    response.set_index(["date"], inplace = True)
    response = response[historical_date:last_date]["HIGH"]
    response = response.rename("VIX_high")
    
    return response

In [4]:
import time
import datetime
from datetime import date
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os

def download_tables(last_date = date.today(), historical_days = 1450):
    '''
    Downloads 10-Y bond historical daily data
    '''
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get('https://finance.yahoo.com/quote/%5ETNX/history?period1=1492732800&period2=1650326400&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true')
    
    #remove old file with data if it's available
    filePath = 'C:\\Users\\oleg.kazanskyi\\Downloads\\^TNX.csv'
            
    # Check whether the  specified file is available and remove it  
    if os.path.exists(filePath):
        os.remove(filePath)
        print("We remove old csv file with metadata and replace with the newer one")
    
    #Calculating current and historical periods
    todays_year = last_date.year
    todays_month = last_date.month
    todays_day = last_date.day
    historical_date = last_date-datetime.timedelta(days=historical_days)
    historical_year = historical_date.year
    historical_month = historical_date.month
    historical_day = historical_date.day
    
    #Searching an input field for dates
    elem = driver.find_element_by_xpath('//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/div[1]/div/div/div/span')
    elem.click()
    time.sleep(3)
    
    #Enter historical date
    elem = driver.find_element_by_xpath('//*[@id="dropdown-menu"]/div/div[1]/input')
    elem.send_keys(historical_year)
    elem.send_keys(Keys.TAB)
    elem.send_keys(historical_month)
    elem.send_keys(historical_day)
    time.sleep(3)
    
    #Enter last date
    elem = driver.find_element_by_xpath('//*[@id="dropdown-menu"]/div/div[2]/input')
    elem.send_keys(todays_year)
    elem.send_keys(Keys.TAB)
    elem.send_keys(todays_month)
    elem.send_keys(todays_day)
    time.sleep(3)
    
    #Press done after entering dates
    elem = driver.find_element_by_xpath('//*[@id="dropdown-menu"]/div/div[3]/button[1]')
    elem.click()
    time.sleep(3)
    
    #Press apply dates range
    elem = driver.find_element_by_xpath('//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[1]/button')
    elem.click()
    time.sleep(3)
    
    #press download data
    elem = driver.find_element_by_xpath('//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[1]/div[2]/span[2]/a/span')
    elem.click()
    time.sleep(3)
    
    driver.close
    
    data = pd.read_csv("C:\\Users\\oleg.kazanskyi\\Downloads\\^TNX.csv")
    
    data.Date = pd.to_datetime(data.Date)
    data.rename(columns = {"Date":"date", "Adj Close":"10Y_bonds"}, inplace = True)
    data.set_index(["date"], inplace = True)
    data = data["10Y_bonds"]
    data.dropna(axis = 0, inplace = True)
    
    return data

In [5]:
import datetime
from datetime import date
import pandas as pd
import numpy as np

def get_dates():
    last_date = date.today()
    historical_days = 1450
    historical_date = last_date-datetime.timedelta(days=historical_days)
    
    dates_df=pd.DataFrame()
    dates_df["date"] = pd.date_range(start=historical_date, end=last_date)
    dates_df["last_crisis_day"] = np.nan
    dates_df.loc[(dates_df['date'] == pd.Timestamp(2018, 12, 21)) | (dates_df['date'] == pd.Timestamp(2020, 3, 13)) | (dates_df['date'] == pd.Timestamp(2022, 4, 29)), 'last_crisis_day'] = dates_df['date']
    dates_df.sort_values(by = 'date', axis = 0, ascending = True, inplace = True)
    dates_df.ffill(axis = 0, inplace = True)
    dates_df.sort_values(by = 'date', axis = 0, ascending = False, inplace = True)
    dates_df["days_after_crisis"] = dates_df["date"] - dates_df["last_crisis_day"] 
    dates_df.set_index(["date"], inplace = True)
    dates_df['days_after_crisis'] = pd.to_numeric(dates_df['days_after_crisis'].dt.days, downcast='integer')
    dates_df.drop(["last_crisis_day"], axis = 1, inplace = True)
    return dates_df

In [6]:
import time
from datetime import date
import pandas as pd
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import io
  
#symbol = "AXP"

def search_symbol(symbol, driver):
    elem = driver.find_element_by_id("ticker")
    elem.send_keys(symbol)
    elem.send_keys(Keys.RETURN)
    driver.implicitly_wait(3)

def find_estim_tables(driver):
    '''
    get a table with earnings on a page  ZACK
    '''
    #Find required table
    elem = driver.find_element_by_xpath('//*[@id="earnings_announcements_earnings_table"]') 
    #get all rows from the table
    rows = [row.text.encode("utf8") for row in elem.find_elements_by_tag_name('tr')]
    #convert list of bytes to list of strings
    rows = [row.decode("utf-8").replace("\n"," ") for row in rows]
    #convert list of strings to a dataframe 
    earnings_history_df = pd.read_csv(io.StringIO('\n'.join(rows)), delim_whitespace=True, header = 0, names = ['date', 'Period_Ending', 'Estimate', 'Reported', 'Surprise', 'Surprise_%', "str1", "str2"])
    
    #Find the latest estimate 
    elem = driver.find_element_by_xpath('//*[@id="right_content"]/section[2]/div') 
    #get all rows from the table
    rows1 = [row.text.encode("utf8") for row in elem.find_elements_by_tag_name('td')]
    rows2 = [row.text.encode("utf8") for row in elem.find_elements_by_tag_name('th')]
    #convert list of bytes to list of strings
    rows1 = [row.decode("utf-8").replace("\n"," ") for row in rows1]
    rows2 = [row.decode("utf-8").replace("\n"," ") for row in rows2]    
    #convert list of strings to a dataframe 
    earnings_latest_df = pd.DataFrame(rows1).transpose()
    earnings_latest_df.columns =  ['Period_Ending', 'Estimate', 'Surprise_%']
    earnings_latest_df["date"] = rows2[-1].split(" ")[0]
    
    earnings = pd.concat([earnings_latest_df, earnings_history_df], ignore_index = True, sort = False)
    earnings['date'] = pd.to_datetime(earnings['date'])
    earnings.set_index(["date"], inplace = True)
    
    return earnings

def find_divid_tables(driver):
    '''
    get a table with dividends on a page  ZACK
    '''
    element = driver.find_element_by_xpath('//*[@id="earnings_announcements_tabs"]/ul')
    driver.execute_script('arguments[0].scrollIntoView({block: "center", inline: "center"})', element)
    
    elem = driver.find_element_by_xpath('//*[@id="ui-id-7"]')
    elem.click()
    time.sleep(3)
    #Expand 100 records
    dropdown = driver.find_element_by_name("earnings_announcements_dividends_table_length")
    Select(dropdown).select_by_visible_text("100")
    
    #Find required table
    elem = driver.find_element_by_xpath('//*[@id="earnings_announcements_dividends_table"]') 
    #get all rows from the table
    rows = [row.text.encode("utf8") for row in elem.find_elements_by_tag_name('tr')]
    #convert list of bytes to list of strings
    rows = [row.decode("utf-8").replace("\n"," ") for row in rows]
    #convert list of strings to a dataframe 
    
    dividends_history_df = pd.read_csv(io.StringIO('\n'.join(rows)), delim_whitespace=True, header = 0, names = ['Date_Paid', 'Amount', 'Date_Announced', 'Ex-Dividend_Date'])
    dividends_history_df.dropna(axis = 0, inplace = True)
    dividends_history_df['Date_Paid'] = pd.to_datetime(dividends_history_df['Date_Paid'])
    dividends_history_df['Date_Announced'] = pd.to_datetime(dividends_history_df['Date_Announced'])
    dividends_history_df['Ex-Dividend_Date'] = pd.to_datetime(dividends_history_df['Ex-Dividend_Date'])
    dividends_history_df.rename(columns = {"Date_Announced":"date"}, inplace = True)
    dividends_history_df.set_index(["date"], inplace = True)
    
    return dividends_history_df

def get_earn_and_dividends(symbol):
    '''
    This function launches browser for data load and fetches earnings and dividends data
    '''
    #Start Chrome 
    driver = webdriver.Chrome(ChromeDriverManager().install())
    #Go to the website
    driver.get('https://www.zacks.com/stock/research/CSCO/earnings-calendar')
    
    #Search stock
    search_symbol(symbol, driver)
    
    #expand_100_earnings values:
    dropdown = driver.find_element_by_name("earnings_announcements_earnings_table_length")
    Select(dropdown).select_by_visible_text("100")
    time.sleep(3)
    
    #Get earnings
    earnings = find_estim_tables(driver)
    time.sleep(3)
    
    #Ge dividends
    dividends = find_divid_tables(driver)
    
    #Close browser
    driver.close()
    
    #Transforming Earnings dataframe to a final version
    #Transform string values to numeric
    earnings.replace({"--":np.nan},inplace = True)
    earnings.Reported, earnings.Estimate, earnings["Surprise_%"] = pd.to_numeric(earnings.Reported.str.replace("$","")), pd.to_numeric(earnings.Estimate.str.replace("$","")), pd.to_numeric(earnings["Surprise_%"].str.replace("%","").str.replace(",",""))
    earnings["surprise_%"] = earnings["Surprise_%"]/100
    earnings["date_of_report"] = earnings.index
    #getting expected future earnings change
    earnings["future_estimate"] = earnings.Estimate.shift(1)
    earnings["previous_surprise"] = earnings["surprise_%"].shift(-1)
    earnings["expected_growth"]= (earnings.future_estimate - earnings.Reported)/earnings.Reported
    earnings = earnings[["surprise_%", "expected_growth", "previous_surprise", "date_of_report"]]
    
    #Transforming Dividends dataframe to a final version
    #STR to value
    dividends.replace({"--":np.nan},inplace = True)
    dividends.Amount = pd.to_numeric(dividends.Amount.str.replace("$",""))
    #get date that we later can use to count days after the announcement
    dividends["date_announced"] = dividends.index
    #Getting dividends trend
    dividends["previous_divid"] = dividends.Amount.shift(-1)
    dividends["dividends_change"] = (dividends.Amount - dividends.previous_divid)/dividends.previous_divid
    dividends = dividends[dividends.dividends_change != 0]
    dividends["prev_div_change"] = dividends.dividends_change.shift(-1)
    dividends = dividends[["dividends_change","prev_div_change","date_announced"]]
    
    #earnings = a.copy()
    #dividends = b.copy()
    
    #Match earnings with dates
    #Creating Dates dataframe with all possible dates values
    dates_df=pd.DataFrame()
    dates_df["date"] = pd.date_range(start=earnings.index.min(), end=earnings.index.max())
    #Set dates column as index
    dates_df.set_index(["date"], inplace = True)
    #Creating a dates_earnings dataset where we extrapolate existing quarterly data to daily
    dates_earnings = dates_df.copy()
    dates_earnings = dates_earnings.join(earnings, how = 'left')
    dates_earnings.sort_values(by = 'date', axis = 0, ascending = True, inplace = True)
    dates_earnings.ffill(axis = 0, inplace = True)
    dates_earnings.sort_values(by = 'date', axis = 0, ascending = False, inplace = True)
    dates_earnings["days_after_earn_report"] = dates_earnings.index - dates_earnings["date_of_report"] 
    dates_earnings['days_after_earn_report'] = pd.to_numeric(dates_earnings['days_after_earn_report'].dt.days, downcast='integer')
    dates_earnings.drop(["date_of_report"], axis = 1, inplace = True)
    
    #Match dividends with dates
    #Creating Dates dataframe with all possible dates values
    if dividends.empty:
        dates_dividends = pd.DataFrame(columns = ["days_after_divid_report", "dividends_change","prev_div_change"])
        dates_dividends.index.names = ['date']
    else:
        dates_df=pd.DataFrame()
        dates_df["date"] = pd.date_range(start=dividends.index.min(), end=date.today())
        #Set dates column as index
        dates_df.set_index(["date"], inplace = True)
        #Creating a dates_ividends dataset where we extrapolate existing quarterly data to daily
        dates_dividends = dates_df.copy()
        dates_dividends = dates_dividends.join(dividends, how = 'left')
        dates_dividends.sort_values(by = 'date', axis = 0, ascending = True, inplace = True)
        dates_dividends.ffill(axis = 0, inplace = True)
        dates_dividends.sort_values(by = 'date', axis = 0, ascending = False, inplace = True)
        dates_dividends["days_after_divid_report"] = dates_dividends.index - dates_dividends["date_announced"] 
        dates_dividends['days_after_divid_report'] = pd.to_numeric(dates_dividends['days_after_divid_report'].dt.days, downcast='integer')
        dates_dividends.drop(["date_announced"], axis = 1, inplace = True)
    
    return dates_earnings, dates_dividends

In [7]:
import os
import pandas as pd
import numpy as np
from datetime import date
import VIX
import new_earnings
import tiingo_data as tii
import US_bond
import crisis_dataset

script_location = os.getcwd()

#get VIX Volatility index
vix_df = VIX.get_vix()

#get 10Y_bond index
usa_bond = US_bond.download_tables()

#get last crisis date data
last_crisis = crisis_dataset.get_dates()

#Get all companies sector, industry, location, etc
metadata = tii.fetch_metadata()

# Get earnings esteem
dow30  = ["AXP", "AMGN", "AAPL", "BA", "CAT", "CSCO", "CVX", "GS", "HD", "HON", "IBM", "INTC", "JNJ", "KO", "JPM", "MCD", "MMM", "MRK", "MSFT", "NKE", "PG", "TRV", "UNH", "CRM", "VZ", "V", "WBA", "WMT", "DIS", "DOW"]

todays_date = date.today()

#a stock symbol to get data
for symbol in dow30[:]:
    #How long historical data we need
    historical_dates_range = 1450
    #How long Moving Average we take
    MA_gener_days = 180
    
    # Hitorical Prices Open, Close, Low, High, Volume for historical data
    stock_dataset = tii.fetch_stock(symbol, todays_date, historical_dates_range)
    # Adding a Moving Average and price position over/below MA to the previous dataset
    #data = tii.create_dataset(stock_dataset, MA_gener_days)
    # Getting stock market cap, PE Ratio, PB Ratio
    fundamentals  =   tii.fetch_fundamentals(symbol, todays_date, historical_dates_range)
    # Financial KPIs
    statements =   tii.fetch_statements(symbol, todays_date, historical_dates_range)
    # Get stock sector and industry
    stock_metadata = metadata[metadata.ticker == symbol.lower()][["sector","industry"]].copy()
    # stock dividents & earnings
    stock_earnings, stock_dividends = new_earnings.get_earn_and_dividends(symbol)    
    
    # Combining all stock's data
    big_dataset = tii.combine_tables(stock_dataset, statements, fundamentals, stock_metadata, historical_dates_range)
    big_dataset = big_dataset.join(vix, how = 'left')
    big_dataset.dropna(inplace = True)
    
    #Combining data with US Bonds
    big_dataset = big_dataset.join(usa_bond, how = 'left')
    #Combining data with earnings & dividends info
    big_dataset = big_dataset.join(stock_earnings, how = 'left')
    big_dataset = big_dataset.join(stock_dividends, how = 'left')
    
    big_dataset.sort_values(by = 'date', axis = 0, ascending = False, inplace = True)
    big_dataset["10Y_bond_MoM"] = big_dataset["10Y_bonds"]-big_dataset["10Y_bonds"].shift(-22)
    big_dataset["Debt-to-Equity_Ratio"] = big_dataset["totalAssets"]/big_dataset["totalLiabilities"]
    big_dataset["DividendsYield"] = big_dataset["payDiv"]/big_dataset["marketCap"]
    big_dataset["PayoutRatio"] = big_dataset["payDiv"]/big_dataset["grossProfit"]
    big_dataset["Acc_Rec_Pay_Ration"] = big_dataset["acctRec"]/big_dataset["acctPay"]
    big_dataset["Earnings_per_stock"] = big_dataset["epsDil"]/big_dataset["close"]
    big_dataset["future_30dprice_change"] = (big_dataset["close"].shift(22) - big_dataset["close"])/big_dataset["close"].shift(22)
    #big_dataset = big_dataset[["roe", "longTermDebtEquity", "grossMargin", "revenueQoQ", "rps", "epsQoQ", "piotroskiFScore", "currentRatio", "roa", "profitMargin","peRatio", "pbRatio","trailingPEG1Y","VIX_high","sector","industry","10Y_bonds", "10Y_bond_MoM","Debt-to-Equity_Ratio","DividendsYield","PayoutRatio","Acc_Rec_Pay_Ration","Earnings_per_stock","dividends_change","prev_div_change","days_after_divid_report","surprise_%", "expected_growth", "previous_surprise","days_after_earn_report"]]
    
    #TO DO 
    #CALC Debt-to-Equity_Ratio,	DividendsYield,	Payout ratio, Acc_Rec_Pay_Ration, YoY Debt ratio, YoY eps, YoY investments
    #REPLACE DividendsYield,	Payout ratio by estimates in Future Periods
    
    csv_file = symbol + ".csv"
    path_to_all_data = os. path. join(script_location, "CSVs")
    path_to_all_data = os. path. join(path_to_all_data, csv_file)
    big_dataset.to_csv(path_to_all_data)